In [ ]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #If the line below doesn't work, uncomment this line (make sure to comment the line below); it should help.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

In [ ]:
tf.config.list_physical_devices()

In [ ]:
dataset_path = os.path.join(os.getcwd(), './', 'seeds.txt')

df = pd.read_csv(dataset_path, 
                lineterminator='\n', 
                sep='\t+|\t\t',
                engine='python', 
                header=None,
                names=['area', 'perimeter', 'compactness', 'length of kernel', 'width of kernel', 'asymmetry coefficient', 'length of kernel groove', 'class']
            )

df.info()

In [ ]:
df.head()

In [ ]:
df['class'].unique()

In [ ]:
fig, ax = plt.subplots(figsize=(24, 16), nrows=3, ncols=7)

cnt = 0
for i in range(7):
    for j in range(i + 1, 7):
        curr_ax = ax[cnt // 7, cnt % 7]
        curr_ax.scatter(df[df.columns[i]], df[df.columns[j]], c=df['class'], cmap='viridis', s=30)
        curr_ax.set_xlabel(df.columns[i])
        curr_ax.set_ylabel(df.columns[j])
        cnt += 1

fig.tight_layout()

In [ ]:
for column in df.columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

In [ ]:
df

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(7,)),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid')
    ])

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from gc import callbacks

compile_model = model.compile(optimizer='adam',
                              loss='mse',
                              metrics=['accuracy'])

In [ ]:
labels = (df['class'] - 1).to_numpy().reshape(-1, 1)

model.fit(df.drop('class', axis=1), 
          tf.one_hot(labels, 3),
          epochs=1000,
          callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5)],
          validation_split=0.2)

# model.fit(df.drop('class', axis=1), 
#           tf.one_hot(labels, 3),
#           epochs=1000,
#           validation_split=0.2)

In [ ]:
categorical_labels = keras.utils.to_categorical(df['class'], num_classes=3)

model2 = keras.models.clone_model(model)

model2.compile(optimizer='adam',
               loss='mse',
               metrics=['accuracy'])

model2.fit(df.drop('class', axis=1), 
           categorical_labels,
           epochs=1000,
           callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
           validation_split=0.2)

In [ ]:
model_tensorboard = keras.models.clone_model(model)

model_tensorboard.compile(optimizer='adam',
               loss='mse',
               metrics=['accuracy'])

model_tensorboard.fit(df.drop('class', axis=1), 
           tf.one_hot(labels, 3),
           epochs=1000,
           callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
                      keras.callbacks.ModelCheckpoint(filepath='models/model_tensorboard.keras', save_best_only=True),
                      keras.callbacks.TensorBoard(log_dir='logs')],
           validation_split=0.2)

Run the following command in a seperate Anaconda Prompt terminal:
```
tensorboard --logdir C:\WGU\Courses\ml-examples\logs --bind_all
```

In [ ]:
non_overfitting_model = keras.Sequential([
    keras.Input(shape=(7,)),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid')
])

In [ ]:
non_overfitting_model.summary()

In [ ]:
non_overfitting_model.compile(optimizer='adam',
                              loss='mse',
                              metrics=['accuracy'])

non_overfitting_model.fit(df.drop('class', axis=1),
                          tf.one_hot(labels, 3),
                          epochs=100,
                          validation_split=0.2,
                          callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5),
                                     keras.callbacks.ModelCheckpoint(filepath='models/model_non_overfitting.keras', save_best_only=True),
                                     keras.callbacks.TensorBoard(log_dir='logs')]
                          )

In [ ]:
model = tf.keras.models.load_model('./models/model_non_overfitting.keras')

In [ ]:
batch_input_data = np.array([
    [11.02, 13.0, 0.8189, 5.325, 2.701, 6.735, 5.163],
    [12.01, 14.0, 0.9189, 2.425, 2.801, 6.835, 15.263],
    [11.02, 13, 0.8189, 5.325, 2.701, 1.735, 1.163],
    # Add more samples as needed
])

# Perform inference
batch_predictions = model.predict(batch_input_data)

# Print the batch predictions
print(batch_predictions)

# Get the predicted classes for the batch
batch_predicted_classes = np.argmax(batch_predictions, axis=1)
print(f"Predicted classes: {batch_predicted_classes + 1}")  # Adding 1 to match the original class labels (1, 2, 3)